In [57]:
A1 = 'A'
A2 = 'B'

B1 = 'A'
B2 = 'B'

In [58]:
import gym
import numpy as np
import random
import torch
import time
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [205]:
from google.colab import files

ACTIONS:

*   left = 0
*   down = 1
*   right = 2
*   up  = 3 

In [59]:
#STATE_MATRIX:
np.array((0,1))

array([0, 1])

**Scenario 1**(same Q table)

A1 (A2) is entrance stimulus. 
B1(B2) on left, B2(B1) on right. 

In [61]:
entrance_stimulus = A1
#entrance_stimulus = A2


#### Reward Structure

In [141]:
#A on left, and B on right

def rewarder1(entrance_stimulus, new_state):
  if new_state ==0:
    new_tile = 'A'
  elif new_state == 2:
    new_tile = 'B'
  if entrance_stimulus == new_tile: 
    reward = 100
  else:
    reward = -100
  return reward

#B on left, and A on right

def rewarder2(entrance_stimulus, new_state):
  if new_state ==0:
    new_tile = 'B'
  elif new_state == 2:
    new_tile = 'A'
  if entrance_stimulus == new_tile: 
    reward = 100
  else:
    reward = -100
  return reward


In [142]:
def choose_action(Q, state, epsilon):
  random_for_epsilon = np.random.rand()
  if random_for_epsilon <= epsilon:
    action = random.choice((0,2))
  elif Q[state,2] > Q[state,0]: 
   action = 2
  elif Q[state,0] > Q[state,2]: 
    action = 0
  elif Q[state,0] == Q[state,2]: 
    action = random.choice((0,2))
  return action

In [214]:
reps = 100
num_episodes = 50
steps_total = [] # store number of steps taken in each episode
rewards_total = [] #store reward obtained for each episode
epsilon_total = [] #store epsilon obtained at the end of each episode
terminal_state = []

In [215]:
rewards_per_rep = []
for i in range(reps):

  epsilon = 0.8
  epsilon_final = 0.1
  epsilon_decay = 0.90
  entrance_stimulus ='A'

  gamma = 0.90 # discount factor
  learning_rate = 0.09 #how important is the difference between q-val from q-table and what's observed

  Q= np.random.rand(3,4)/1000

  rewards_per_episode= []
  for i_episode in range(num_episodes):
    
    state = 1
  
    if epsilon > epsilon_final:
            epsilon *= epsilon_decay
  
    while True:
      action = choose_action(Q, state, epsilon)
    
      if action == 0:
        new_state = 0
      else:
        new_state = 2

      reward = rewarder1(entrance_stimulus, new_state)
      rewards_per_episode.append(reward)
      Q[state, action] = (1- learning_rate)*Q[state, action] + (learning_rate)*(reward)  #+ gamma*(np.max(Q1.loc[new_state, [0,2]])))
      
      done = True

      #print('Episode: {} Reward: {} , Epsilon: {}, Entrance_Stimulus: {}, New_State: {}'.format(i_episode,reward, epsilon, entrance_stimulus, new_state ))
       
      break
  rewards_per_rep.append(np.array(rewards_per_episode))

In [235]:
rewards_per_rep = np.array(rewards_per_rep)
y_data = np.mean(rewards_per_rep, axis = 0)
y_std = np.std(rewards_per_rep, axis = 0)
y_max = y_data - y_std/2
y_min = y_data + y_std/2

In [236]:
y_data.shape

(50,)

In [237]:
#1 rep
import plotly.express as px
fig = px.scatter(x=np.arange(0,50), y= rewards_per_rep[2])

fig.update_layout(
    title=" Rewards for a random rep",
    xaxis_title="Episodes",
    yaxis_title="Reward per episode",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ))

fig.update_yaxes(nticks=20) 

fig.show()

In [238]:
fig.write_image("fig1.png")
files.download('fig1.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [240]:
import plotly.graph_objects as go

fig = go.Figure([
    go.Scatter(name = 'reward',
        x=np.arange(0,50),
        y=np.mean(rewards_per_rep, axis = 0),
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=np.arange(0,50),
        y=np.mean(rewards_per_rep, axis = 0) + y_std/2 ,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=np.arange(0,50),
        y=np.mean(rewards_per_rep, axis = 0) - y_std/2,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title= 'Average reward per episode',
    xaxis_title= 'Episodes',
    title='Rewards for 100 reps',
    hovermode="x",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
))
fig.update_yaxes(nticks=20) 
fig.show()


In [241]:
fig.write_image("fig2.png")
files.download('fig2.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Transfer setting (flipped environment)

In [223]:

rewards_per_rep2 = []
for i in range(reps):

  epsilon = 0.8
  epsilon_final = 0.1
  epsilon_decay = 0.90
  entrance_stimulus ='A'

  gamma = 0.90 # discount factor
  learning_rate = 0.09 #how important is the difference between q-val from q-table and what's observed

  Q2 = Q

  rewards_per_episode= []
  for i_episode in range(num_episodes):
    
    state = 1
  
    if epsilon > epsilon_final:
            epsilon *= epsilon_decay
  
    while True:
      action = choose_action(Q, state, epsilon)
    
      if action == 0:
        new_state = 0
      else:
        new_state = 2

      reward = rewarder2(entrance_stimulus, new_state)
      rewards_per_episode.append(reward)
      Q2[state, action] = (1- learning_rate)*Q2[state, action] + (learning_rate)*(reward)  #+ gamma*(np.max(Q1.loc[new_state, [0,2]])))
      
      done = True

      #print('Episode: {} Reward: {} , Epsilon: {}, Entrance_Stimulus: {}, New_State: {}'.format(i_episode,reward, epsilon, entrance_stimulus, new_state ))
       
      break
  rewards_per_rep2.append(np.array(rewards_per_episode))

In [224]:
Q2[1,[0,2]]

array([-100.,  100.])

In [225]:
rewards_per_rep2 = np.array(rewards_per_rep2)
y_data = np.mean(rewards_per_rep2, axis = 0)
y_std = np.std(rewards_per_rep2, axis = 0)
y_max = y_data - y_std/2
y_min = y_data + y_std/2

In [226]:
#1 rep

fig2 = px.scatter(x=np.arange(0,50), y= rewards_per_rep2[80])

fig2.update_layout(
    title=" Rewards for a random rep in transfer setting",
    xaxis_title="Episodes",
    yaxis_title="Reward per episode",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ))

fig2.update_yaxes(nticks=20) 

fig2.show()

In [227]:
fig2.write_image("fig3.png")
files.download('fig3.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [228]:
fig = go.Figure([
    go.Scatter(
        name='Reward',
        x=np.arange(0,50),
        y=np.mean(rewards_per_rep2, axis = 0),
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=np.arange(0,50),
        y=np.mean(rewards_per_rep2, axis = 0) + y_std/2 ,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=np.arange(0,50),
        y=np.mean(rewards_per_rep2, axis = 0) - y_std/2,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title= 'average reward per episode',
    xaxis_title= 'episode',
    title='Rewards for 100 reps - transfer setting',
    hovermode="x",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
))
fig.update_yaxes(nticks=20) 
fig.show()

In [229]:
fig.write_image("fig4.png")
files.download('fig4.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>